<a href="https://colab.research.google.com/github/Apple03244/Colaboratory/blob/main/Model_Save_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import google.colab as gc
files=gc.files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic
! unzip /content/titanic.zip

Saving kaggle.json to kaggle (2).json
titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/titanic.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [62]:
import pandas as pd
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
sub_mission=pd.read_csv('/content/gender_submission.csv')

##Titanic data

In [63]:
target='Survived'

In [64]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [65]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


### Select drop or need to fill Nan value
- 1. Cabin column drop

why? this columns have a lot of Nan value even more than 60%

In [86]:
import sklearn.impute as skimp
import sklearn.neighbors as sknei
import sklearn.model_selection as skmod
import sklearn.ensemble as skens
import sklearn.preprocessing as skpre
import numpy as np
np.random.seed(10)

In [70]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [71]:
drop_columns=["Name","Cabin","Ticket"]

In [69]:
train[target].value_counts()

0    549
1    342
Name: Survived, dtype: int64

- 2. drop data which have Nan value of Embarked

why? there are just 2 data in train data & in test data, their all have this.

In [78]:
train_copy=train.drop(train.loc[train["Embarked"].isna()].index,axis=0)

In [79]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 90.3+ KB


- 3. fill Nan value of column Age

In [80]:
numeric_cols=["Pclass","SibSp","Parch","Fare","Age"]

check KNN method's rmse

why? it seems useful to predict Nan value of Age

In [87]:
import sklearn.ensemble as skens
import sklearn.neighbors as sknei

In [82]:
knn_regressior=sknei.KNeighborsRegressor(n_neighbors=5,weights='distance',n_jobs=3)
nan_regressior=skens.BaggingClassifier(estimator=knn_regressior,n_estimators=5,oob_score=True,n_jobs=3,random_state=10)

In [84]:
knn_train_data=train_copy.drop(train_copy[train_copy["Age"].isna()].index,axis=0)
knn_train_data=skpre.minmax_scale(X=knn_train_data[])
nan_regressior.fit(X=knn_train_data[[x for x in numeric_cols if x!='Age']],y=knn_train_data['Age'])

ValueError: ignored

In [85]:
knn_train_data[[x for x in numeric_cols if x!='Age']]

,Pclass,SibSp,Parch,Fare
0,3,1,0,7.2500
1,1,1,0,71.2833
2,3,0,0,7.9250
3,1,1,0,53.1000
4,3,0,0,8.0500
...,...,...,...,...
885,3,0,5,29.1250
886,2,0,0,13.0000
887,1,0,0,30.0000
889,1,0,0,30.0000
